# County Business Patterns Data Aggregation 2000-23

In [ ]:
# Import statemants
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, min, max, count, round, lit, when
from pyspark.sql.types import IntegerType

# Create Spark environment
spark = SparkSession.builder.appName("CountyBusinessPatterns").getOrCreate()

sc = spark.sparkContext

### Load Data

In [ ]:
# Upload data
from google.colab import files
uploaded = files.upload()

Saving zbp00detail.txt to zbp00detail.txt
Saving zbp01detail.txt to zbp01detail.txt
Saving zbp02detail.txt to zbp02detail.txt
Saving zbp03detail.txt to zbp03detail.txt
Saving zbp04detail.txt to zbp04detail.txt
Saving zbp05detail.txt to zbp05detail.txt
Saving zbp06detail.txt to zbp06detail.txt
Saving zbp07detail.txt to zbp07detail.txt
Saving zbp08detail.txt to zbp08detail.txt
Saving Zbp09detail.txt to Zbp09detail.txt
Saving zbp10detail.txt to zbp10detail.txt
Saving zbp11detail.txt to zbp11detail.txt
Saving zbp12detail.txt to zbp12detail.txt
Saving zbp13detail.txt to zbp13detail.txt
Saving zbp14detail.txt to zbp14detail.txt
Saving zbp15detail.txt to zbp15detail.txt
Saving zbp16detail.txt to zbp16detail.txt
Saving zbp17detail.txt to zbp17detail.txt
Saving zbp18detail.txt to zbp18detail.txt
Saving zbp19detail.txt to zbp19detail.txt
Saving zbp20detail.txt to zbp20detail.txt
Saving zbp21detail.txt to zbp21detail.txt
Saving zbp22detail.txt to zbp22detail.txt
Saving zbp23detail.txt to zbp23det

In [ ]:
# Upload data
from google.colab import files
uploaded = files.upload()

Saving geonames-postal-code.csv to geonames-postal-code.csv


### Data Processing

In [ ]:
 # Define File and Schema Mappings
file_map = {
    2000: "zbp00detail.txt", 2001: "zbp01detail.txt", 2002: "zbp02detail.txt",
    2003: "zbp03detail.txt", 2004: "zbp04detail.txt", 2005: "zbp05detail.txt",
    2006: "zbp06detail.txt", 2007: "zbp07detail.txt", 2008: "zbp08detail.txt",
    2009: "zbp09detail.txt", 2010: "zbp10detail.txt", 2011: "zbp11detail.txt",
    2012: "zbp12detail.txt", 2013: "zbp13detail.txt", 2014: "zbp14detail.txt",
    2015: "zbp15detail.txt", 2016: "zbp16detail.txt", 2017: "zbp17detail.txt",
    2018: "zbp18detail.txt", 2019: "zbp19detail.txt", 2020: "zbp20detail.txt",
    2021: "zbp21detail.txt", 2022: "zbp22detail.txt", 2023: "zbp23detail.txt"
}

In [ ]:
# Define Standardized Columns to Keep
TARGET_COLUMNS = ["ZIP", "NAICS","EstablishmentCount","Year"]

#### Cleaning Functions to Handle Schema Differences


In [ ]:
from pyspark.sql.functions import col, lit, upper, substring

# Cleaning Function
def clean_df(file_name, year):

    df = spark.read.csv(f"{file_name}", header=True, inferSchema=True)

    # Standardize columns
    df = df.select(
        upper(col("zip")).alias("ZIP"),
        col("naics").alias("NAICS"),
        col("est").alias("EstablishmentCount"),
        lit(year).alias("Year")
    )
    return df

In [ ]:
# List to hold cleaned DataFrames
cleaned_dfs = []

# read each file from file map
for year, file_name in file_map.items():
  cleaned_df = clean_df(file_name, year)

  cleaned_dfs.append(cleaned_df)

#### Data Cleaning

In [ ]:
# Start with the first DataFrame in the list
cbp_panel_df = cleaned_dfs[0]

# Iteratively union the rest of the DataFrames
for next_df in cleaned_dfs[1:]:
    cbp_panel_df = cbp_panel_df.union(next_df)


In [ ]:
# Read file for Zip code loaction data
postal_code_df = spark.read.csv("geonames-postal-code.csv", header=True, inferSchema=True)

In [ ]:
# Standardize columns
postal_code_df = postal_code_df.select(
    col("postal code").alias("ZIP").cast("string"),
    col("latitude").alias("latitude"),
    col("longitude").alias("longitude")
).distinct()

In [ ]:
#join data
az_zbp_panel_df = cbp_panel_df.join(
    postal_code_df,
    on="ZIP",
    how="inner"
)

In [ ]:
# write to file
output_path = "Combined_CBPData_2000-23.csv"
az_zbp_panel_df.coalesce(1).write.csv(
    path=output_path,
    header=True,
    mode="overwrite"
)